In [1]:
from pyquery import PyQuery as pq
import re
import pandas as pd

In [2]:
def flatten(arr):
    return [val for sublist in arr for val in sublist]
def clean_term(term):
    term = re.sub("</div>\n<div>", "\n", term)
    term = re.sub("\xa0", "", term)
    term = re.sub("\*", "", term)
    term = re.sub("</div>\n<div>", "\n", term)
    term = re.sub("(</div>)|(<div>)", "", term)
    term = re.sub("<span.*>.*<\/span>", "", term)
    term = re.sub("<b>.*<\/b>", "", term)
    term = re.sub("<br\/*>", "\n", term)
    return term.strip()
def fetch_terms(filename):
    with open("data/"+filename) as spine:
        doc = pq(spine.read())
        terms = [clean_term(row("td").eq(0).html()) for row in list(doc("tr").items())[1:]]
        terms = flatten([term.split("\n") for term in terms if term != ""])
        return terms
def gen_frame(filename, lesson):
    terms = fetch_terms(filename)
    return pd.DataFrame({"Term": terms, "Lesson": [lesson]*len(terms)})

In [3]:
pd.concat([
    gen_frame("spine.html", "Spine"),
    gen_frame("thoracic_wall.html", "Thoracic Wall")
]).to_csv("terms.csv", index=False)